In [1]:
import torch
device = torch.device('cuda:0')
print(f"Using device: {device}")

Using device: cuda:0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


installing dependencies for the vision transformer models

In [3]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 548 kB 37.7 MB/s 
     |████████████████████████████████| 163 kB 69.2 MB/s 


In [4]:
import cv2
import os
import shutil
import torchvision
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import timm
from torchvision import datasets, models, transforms
from torchvision.models import alexnet, inception_v3, resnet18
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from torchvision.io import read_image
from torch.utils.data import Dataset, SubsetRandomSampler, DataLoader, ConcatDataset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import torch.nn.functional as func
from torchvision.transforms import transforms
import warnings

warnings.filterwarnings('ignore')

# Dataset Preparation

Function for normalizing images using the "Stain Normalization Technique" found in literature

In [ ]:
"""
Acknowledgements:

Copyright (c) 2019, Mitko Veta(1), Geoffrey F. Schau(2)
1 Image Sciences Institute
University Medical Center
Utrecht, The Netherlands

2 Biomedical Engineering Department
Oregon Health & Science University
Portland, OR, USA

Original Python Code: https://github.com/schaugf/HEnorm_python

Function based on the algorithm in the following paper:
 method for normalizing histology slides for quantitative analysis. 
M. Macenko et al., ISBI 2009
    http://wwwx.cs.unc.edu/~mn/sites/default/files/macenko2009.pdf

"""

# Default values taken from literature

"""
Io -> Transmitted light intensity, normalizing factor for image intensities
alpha -> Tolerance for the pseudo-min and pseudo-max (default: 1)
beta -> OD threshold for transparent pixels (default: 0.15)
HERef -> Reference H&E Optical Density (OD) matrix
maxCRef -> Reference maximum stain concentrations for H&E
"""
Io = 240
alpha = 1
beta = 0.15
HERef = np.array([[0.5626, 0.2159], [0.7201, 0.8012], [0.4062, 0.5581]])
maxCRef = np.array([1.9705, 1.0308])

def normalize_image(img_name, img_folder, save_folder):

    # Step 1 - Convert RGB to OD
    image = cv2.imread(f'{img_folder}/{img_name}', 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    h,w,c = image.shape
    image = image.reshape((-1,3))


    OD = -np.log((image.astype(np.float)+1)/Io)

    #  Step 2 - Remove data with OD intensity less than β
    ODhat = OD[~np.any(OD < beta, axis=1)]

    # Step 3 - Calculate Single Value Decompostion (SVD) on the OD tuples
    eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))

    # Step 4 - Create plane from the SVD directions with the two largest values 
    That = ODhat.dot(eigvecs[:,1:3])

    # Step 5 - Project data onto the plane, and normalize to unit length
    # Step 6 - Calculate angle of each point wrt the first SVD direction

    phi = np.arctan2(That[:,1],That[:,0])

    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)

    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)

    if vMin[0] > vMax[0]:    
        HE = np.array((vMin[:,0], vMax[:,0])).T
    
    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T
    
    Y = np.reshape(OD, (-1, 3)).T
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]

    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])

    # Step 8 - Convert extreme values back to OD space
    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)

    plt.imsave(f'{save_folder}/{img_name}', Inorm)
                  
                  


Function for moving images into its desired folders

In [ ]:
def move_image(df, src, dest):
    training_arr = df.to_numpy()

    for i in training_arr:
        if i[1] == 'benign':
            os.replace(src=f'{src}/{i[0]}', dst=f'{dest}/benign/{i[0]}')
        else:
            os.replace(src=f'{src}/{i[0]}', dst=f'{dest}/malignant/{i[0]}')

### Normalizing the images

In [ ]:
img40_df = pd.DataFrame(os.listdir('Images_40x/'))
img100_df = pd.DataFrame(os.listdir('Images_100x/'))
img200_df = pd.DataFrame(os.listdir('Images_200x/'))
img400_df = pd.DataFrame(os.listdir('Images_400x/'))

for i in img40_df[0]:
    normalize_image(i, 'Images_40X', 'Normalized_40x')

for j in img100_df[0]:
    normalize_image(j, 'Images_100X', 'Normalized_100x')

for k in img200_df[0]:
    normalize_image(k, 'Images_200X', 'Normalized_200x')

for l in img400_df[0]:
    normalize_image(l, 'Images_400X', 'Normalized_400x')

/var/folders/nz/9k3gxmx13gvdm47n00h2sy080000gn/T/ipykernel_15140/3824165961.py:47: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  OD = -np.log((image.astype(np.float)+1)/Io)


### Organizing the images into training data and testing data

#### Images with 40x magnification

**Creating a dataframe for the images at 40x magnification**

In [ ]:
image40_df = pd.DataFrame(os.listdir('Normalized_40X'))
image40_df = image40_df.rename(columns={0:'image name'})
image40_df['label'] = image40_df['image name'].apply(lambda x:x.split("_")[0])
image40_df['class'] = image40_df['label'].apply(lambda x: 0 if x =='benign' else 1)

image40_df.head(10)

,image name,label,class
0,benign_SOB_B_F-14-23060CD-40-008.png,benign,0
1,benign_SOB_B_F-14-23060AB-40-022.png,benign,0
2,benign_SOB_B_PT-14-21998AB-40-021.png,benign,0
3,benign_SOB_B_PT-14-21998AB-40-035.png,benign,0
4,benign_SOB_B_F-14-21998CD-40-005.png,benign,0
5,malignant_SOB_M_DC-14-14015-40-009.png,malignant,1
6,benign_SOB_B_PT-14-21998AB-40-009.png,benign,0
7,benign_SOB_B_F-14-21998CD-40-011.png,benign,0
8,benign_SOB_B_TA-14-16184CD-40-031.png,benign,0
9,malignant_SOB_M_MC-14-13413-40-028.png,malignant,1


Splitting the data into a 90-10 split.

In [ ]:
train40_df, test40_df = train_test_split(image40_df, test_size=0.1, train_size=0.9)

train40_df.head()

,image name,label,class
1050,malignant_SOB_M_DC-14-8168-40-008.png,malignant,1
53,benign_SOB_B_PT-14-22704-40-021.png,benign,0
1943,malignant_SOB_M_DC-14-16716-40-001.png,malignant,1
1882,malignant_SOB_M_LC-14-13412-40-028.png,malignant,1
1880,malignant_SOB_M_DC-14-18650-40-029.png,malignant,1


Testing data

In [ ]:
test40_df.head()

,image name,label,class
664,malignant_SOB_M_DC-14-6241-40-026.png,malignant,1
1187,benign_SOB_B_F-14-14134E-40-023.png,benign,0
190,malignant_SOB_M_MC-14-19979-40-004.png,malignant,1
507,malignant_SOB_M_DC-14-11520-40-021.png,malignant,1
1033,malignant_SOB_M_DC-14-13412-40-008.png,malignant,1


**Creating subfolders so that datasets can easily be created and loaded**

In [ ]:
os.mkdir('Normalized_40x/training')
os.mkdir('Normalized_40x/testing')
os.mkdir('Normalized_40x/training/benign')
os.mkdir('Normalized_40x/training/malignant')
os.mkdir('Normalized_40x/testing/benign')
os.mkdir('Normalized_40x/testing/malignant')

"""

os.mkdir('Normalized_200x/training')
os.mkdir('Normalized_200x/testing')
os.mkdir('Normalized_200x/training/benign')
os.mkdir('Normalized_200x/training/malignant')
os.mkdir('Normalized_200x/testing/benign')
os.mkdir('Normalized_200x/testing/malignant')

os.mkdir('Normalized_200x/training')
os.mkdir('Normalized_200x/testing')
os.mkdir('Normalized_200x/training/benign')
os.mkdir('Normalized_200x/training/malignant')
os.mkdir('Normalized_200x/testing/benign')
os.mkdir('Normalized_200x/testing/malignant')"""

"os.mkdir('Normalized_100x/training')\nos.mkdir('Normalized_100x/testing')\nos.mkdir('Normalized_100x/training/benign')\nos.mkdir('Normalized_100x/training/malignant')\nos.mkdir('Normalized_100x/testing/benign')\nos.mkdir('Normalized_100x/testing/malignant')\n\nos.mkdir('Normalized_200x/training')\nos.mkdir('Normalized_200x/testing')\nos.mkdir('Normalized_200x/training/benign')\nos.mkdir('Normalized_200x/training/malignant')\nos.mkdir('Normalized_200x/testing/benign')\nos.mkdir('Normalized_200x/testing/malignant')\n\nos.mkdir('Normalized_200x/training')\nos.mkdir('Normalized_200x/testing')\nos.mkdir('Normalized_200x/training/benign')\nos.mkdir('Normalized_200x/training/malignant')\nos.mkdir('Normalized_200x/testing/benign')\nos.mkdir('Normalized_200x/testing/malignant')"

Moving images to its respective folders

In [ ]:
move_image(df=train40_df, src='Normalized_40X', dest='Normalized_40X/training')
move_image(df=test40_df,src='Normalized_40X', dest='Normalized_40X/testing')

#### Images with 100x magnification

In [ ]:
image_df = pd.DataFrame(os.listdir('Normalized_100x'))
image_df = image_df.rename(columns={0:'image name'})
image_df['label'] = image_df['image name'].apply(lambda x:x.split("_")[0])
image_df['class'] = image_df['label'].apply(lambda x: 0 if x =='benign' else 1)

In [ ]:
train_df, test_df = train_test_split(image_df, test_size=0.1, train_size=0.9)

In [ ]:
os.mkdir('Normalized_100x/training')
os.mkdir('Normalized_100x/testing')
os.mkdir('Normalized_100x/training/benign')
os.mkdir('Normalized_100x/training/malignant')
os.mkdir('Normalized_100x/testing/benign')
os.mkdir('Normalized_100x/testing/malignant')

In [ ]:
move_image(df=train_df, src='Normalized_100x', dest='Normalized_100x/training')

In [ ]:
move_image(df=test_df,src='Normalized_100x', dest='Normalized_100x/testing')

#### Images with 200x magnification


In [ ]:
image_df = pd.DataFrame(os.listdir('Normalized_200x'))
image_df = image_df.rename(columns={0:'image name'})
image_df['label'] = image_df['image name'].apply(lambda x:x.split("_")[0])
image_df['class'] = image_df['label'].apply(lambda x: 0 if x =='benign' else 1)

In [ ]:
train_df, test_df = train_test_split(image_df, test_size=0.1, train_size=0.9)

In [ ]:
os.mkdir('Normalized_200x/training')
os.mkdir('Normalized_200x/testing')
os.mkdir('Normalized_200x/training/benign')
os.mkdir('Normalized_200x/training/malignant')
os.mkdir('Normalized_200x/testing/benign')
os.mkdir('Normalized_200x/testing/malignant')

In [ ]:
move_image(df=train_df, src='Normalized_200x', dest='Normalized_200x/training')

In [ ]:
move_image(df=test_df,src='Normalized_200x', dest='Normalized_200x/testing')

#### Images with 400x magnification

In [ ]:
image_df = pd.DataFrame(os.listdir('Normalized_400x'))
image_df = image_df.rename(columns={0:'image name'})
image_df['label'] = image_df['image name'].apply(lambda x:x.split("_")[0])
image_df['class'] = image_df['label'].apply(lambda x: 0 if x =='benign' else 1)

In [ ]:
train_df, test_df = train_test_split(image_df, test_size=0.1, train_size=0.9)

In [ ]:
os.mkdir('Normalized_400x/training')
os.mkdir('Normalized_400x/testing')
os.mkdir('Normalized_400x/training/benign')
os.mkdir('Normalized_400x/training/malignant')
os.mkdir('Normalized_400x/testing/benign')
os.mkdir('Normalized_400x/testing/malignant')

In [ ]:
move_image(df=train_df, src='Normalized_400x', dest='Normalized_400x/training')

In [ ]:
move_image(df=test_df,src='Normalized_400x', dest='Normalized_400x/testing')

# Model Training and Evaluation

This function sets the "requires.grad" attribute for all parameters to be false. This helps to freeze all the layers so that the model can be used as a fixed feature extractor. 

In [5]:
def set_parameter_requires_grad(model, is_fixed_feature_extractor):
    if is_fixed_feature_extractor:
        for param in model.parameters():
            param.requires_grad = False

This function helps to reshape the networks that are being used in the project. For each network, the final classification layers are adjusted for the number of classes being classified. Since the ImageNet dataset has 1000 classes, we must reshape the networks for this project's classes (i.e. benign and malignant). This model can also return the input size for each model so data augmentation can take place easier. 

In [6]:
def create_model(name, num_classes, is_fixed_feature_extractor=False):
    model_ft = None

    if name == 'AlexNet':
        model_ft = alexnet(pretrained = True)
        set_parameter_requires_grad(model_ft, is_fixed_feature_extractor)
        num_features = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_features,num_classes)
        
    
    elif name == 'Inception':
        model_ft = inception_v3(pretrained = True)
        set_parameter_requires_grad(model_ft, is_fixed_feature_extractor)
        num_features = model_ft.AuxLogits.fc.in_features # NB: Inception has auxillary outputs
        model_ft.AuxLogits.fc == nn.Linear(num_features, num_classes)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        
    
    elif name == 'ResNet':
        model_ft = resnet18(pretrained = True)
        set_parameter_requires_grad(model_ft, is_fixed_feature_extractor)
        num_features = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_features, num_classes)

    elif name == 'Transformer':
      model_ft = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
      set_parameter_requires_grad(model_ft, is_fixed_feature_extractor)  
    
    return model_ft

Function for training the models

In [7]:
def train_model(model, dataloader, criterion, optimizer, is_inception = False):
    model.train()

    for batch_idx, (input, targets)  in enumerate(dataloader):
        inputs = input.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            # Accounting for the inception v3's auxillary outputs
            if is_inception:
                outputs, aux_outputs = model(inputs)
                loss_main = criterion(outputs, targets)
                loss_aux = criterion(aux_outputs, targets)
                loss = loss_main + (0.4 * loss_aux) # 0.4 is the weight for auxillary classifier
            
            else:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
            
            _, preds = torch.max(outputs, 1) 
            loss.backward()
            optimizer.step()

Function for validating the trained model

In [8]:
def validate_model(model, dataloader, criterion):
    model.eval()
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            bs, ncrops, c, h, w = np.shape(inputs)
            inputs = inputs.view(-1, c, h, w)
            inputs = inputs.to(device)
            targets = targets.to(device)

            
            outputs = model(inputs)
            outputs_avg = outputs.view(bs,ncrops, -1).mean(1)
            loss = criterion(outputs_avg, targets)
        
            


Function for evaluating the model after training

In [9]:
def model_eval(model, dataloader):
    c_mat = 0
    model.eval()

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            bs, ncrops, c, h, w = np.shape(inputs)
            inputs = inputs.view(-1, c, h, w)
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            outputs_avg = outputs.view(bs,ncrops, -1).mean(1)
            _, preds = torch.max(outputs_avg,1)
            preds = preds.to(device)

            c_mat += confusion_matrix(targets.cpu(),preds.cpu(),labels=[0,1])
        
        true_neg = c_mat[0][0]
        true_positive = c_mat[1][1]
        false_positive = c_mat[0][1]
        false_negative = c_mat[1][0]

        accuracy = np.sum(np.diag(c_mat)/np.sum(c_mat))
        sensitivity = true_positive / (true_positive+false_negative)
        precision = true_positive/(true_positive+false_positive)
        specificity = true_neg / (true_neg+false_positive) 
        F1 = (2*sensitivity*precision)/(sensitivity+precision)

        return accuracy, sensitivity, precision, specificity, F1
            

Custom Dataset class for data augmentation

In [10]:
class WrapperDataset:
    def __init__(self, dataset, transform=None, target_transform=None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        image, label = self.dataset[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.dataset)

## AlexNet models

Data Augmentation dimensions specific to this model

In [ ]:
data_transforms = {
    'train_transform' : transforms.Compose(
        [
            transforms.Resize(224),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ]
    ),

    'test_transform' : transforms.Compose(
        [
            transforms.TenCrop(224),
            transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
        ]
    )
}

### Working with images at 40x magnification

Loading the model and datasets

In [ ]:
alexnet_ft = create_model(name='AlexNet',num_classes=2)
alexnet_ft = alexnet_ft.to(device)

data_dir = 'Normalized_40X'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fine tuning the model using K-Fold cross validation

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(alexnet_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(alexnet_ft, train_loader, criterion, optimizer)
            validate_model(alexnet_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(alexnet_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")
           
       

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

Creating a model using AlexNet as a fixed feature extractor

In [ ]:
alexnet_ffe = create_model(name='AlexNet',num_classes= 2, is_fixed_feature_extractor=True)
alexnet_ffe = alexnet_ffe.to(device)

Training the AlexNet model

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(alexnet_ffe.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(alexnet_ffe, train_loader, criterion, optimizer)
            validate_model(alexnet_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(alexnet_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")
           

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 100x magnification

Creating the model

In [ ]:
alexnet_ft = create_model(name='AlexNet',num_classes=2)
alexnet_ft = alexnet_ft.to(device)

data_dir = 'Normalized_100x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fine tuning the AlexNet model

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(alexnet_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(alexnet_ft, train_loader, criterion, optimizer)
            validate_model(alexnet_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(alexnet_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

Creating an AlexNet model thats a fixed feature extractor

In [ ]:
alexnet_ffe = create_model(name='AlexNet',num_classes= 2, is_fixed_feature_extractor=True)
alexnet_ffe = alexnet_ffe.to(device)

/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training the AlexNet feature extractor model

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(alexnet_ffe.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(alexnet_ffe, train_loader, criterion, optimizer)
            validate_model(alexnet_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(alexnet_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 200x magnification

In [ ]:
alexnet_ft = create_model(name='AlexNet',num_classes=2)
alexnet_ft = alexnet_ft.to(device)

data_dir = 'Normalized_200x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(alexnet_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(alexnet_ft, train_loader, criterion, optimizer)
            validate_model(alexnet_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(alexnet_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

In [ ]:
alexnet_ffe = create_model(name='AlexNet',num_classes= 2, is_fixed_feature_extractor=True)
alexnet_ffe = alexnet_ffe.to(device)

/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(alexnet_ffe.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(alexnet_ffe, train_loader, criterion, optimizer)
            validate_model(alexnet_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(alexnet_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 400x magnification

In [ ]:
alexnet_ft = create_model(name='AlexNet',num_classes=2)
alexnet_ft = alexnet_ft.to(device)

data_dir = 'Normalized_400x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(alexnet_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(alexnet_ft, train_loader, criterion, optimizer)
            validate_model(alexnet_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(alexnet_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

In [ ]:
alexnet_ffe = create_model(name='AlexNet',num_classes= 2, is_fixed_feature_extractor=True)
alexnet_ffe = alexnet_ffe.to(device)

/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/brennonmaistry/UKZN/UKZN_py-env/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(alexnet_ffe.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(alexnet_ffe, train_loader, criterion, optimizer)
            validate_model(alexnet_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(alexnet_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

## Inception models

Data Augmentations specific to this model

In [ ]:
data_transforms = {
    'train_transform' : transforms.Compose(
        [
            transforms.Resize(299),
            transforms.RandomResizedCrop(299),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ]
    ),

    'test_transform' : transforms.Compose(
        [
            transforms.TenCrop(299),
            transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
        ]
    )
}

### Working with images at 40x magnification

Loading the datasets 

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_40x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

Fine tuning the model

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    inception_ft = create_model('Inception', 2)
    inception_ft = inception_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(inception_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(inception_ft, train_loader, criterion, optimizer, is_inception=True)
            validate_model(inception_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(inception_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

Training the model as a fixed feature extractor

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    inception_ffe = create_model('Inception', 2, is_fixed_feature_extractor=True)
    inception_ffe = inception_ffe.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    params_to_update = []
    for name,param in inception_ffe.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            

    optimizer = optim.SGD(params_to_update, lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(inception_ffe, train_loader, criterion, optimizer, is_inception=True)
            validate_model(inception_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(inception_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 100x magnification

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_100x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    inception_ft = create_model('Inception', 2)
    inception_ft = inception_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(inception_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(inception_ft, train_loader, criterion, optimizer, is_inception=True)
            validate_model(inception_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(inception_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    inception_ffe = create_model('Inception', 2, is_fixed_feature_extractor=True)
    inception_ffe = inception_ffe.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    params_to_update = []
    for name,param in inception_ffe.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            

    optimizer = optim.SGD(params_to_update, lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(inception_ffe, train_loader, criterion, optimizer, is_inception=True)
            validate_model(inception_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(inception_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 200x magnificiation

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_200x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']

data2 = image_datasets['testing']

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    inception_ft = create_model('Inception', 2)
    inception_ft = inception_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(inception_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(inception_ft, train_loader, criterion, optimizer, is_inception=True)
            validate_model(inception_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(inception_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
E

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    inception_ffe = create_model('Inception', 2, is_fixed_feature_extractor=True)
    inception_ffe = inception_ffe.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    params_to_update = []
    for name,param in inception_ffe.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            

    optimizer = optim.SGD(params_to_update, lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(inception_ffe, train_loader, criterion, optimizer, is_inception=True)
            validate_model(inception_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(inception_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 400x magnification

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_400x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    inception_ft = create_model('Inception', 2)
    inception_ft = inception_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(inception_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(inception_ft, train_loader, criterion, optimizer, is_inception=True)
            validate_model(inception_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(inception_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    inception_ffe = create_model('Inception', 2, is_fixed_feature_extractor=True)
    inception_ffe = inception_ffe.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    params_to_update = []
    for name,param in inception_ffe.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            

    optimizer = optim.SGD(params_to_update, lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(inception_ffe, train_loader, criterion, optimizer, is_inception=True)
            validate_model(inception_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(inception_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
E

## ResNet models


Data Augmentation with dimensions specific to this model

In [ ]:
data_transforms = {
    'train_transform' : transforms.Compose(
        [
            transforms.Resize(224),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ]
    ),

    'test_transform' : transforms.Compose(
        [
            transforms.TenCrop(224),
            transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
        ]
    )
}

### Working with images at 40x magnification

Dataset preparation

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_40x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

Fine tuning the ResNet model

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    resnet_ft = create_model('ResNet', 2)
    resnet_ft = resnet_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(resnet_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(resnet_ft, train_loader, criterion, optimizer)
            validate_model(resnet_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(resnet_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
E

Training the model as a fixed feature extractor

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    resnet_ffe = create_model('ResNet', 2, is_fixed_feature_extractor=True)
    resnet_ffe = resnet_ffe.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)
    params_to_update = []
    for name,param in resnet_ffe.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)

    optimizer = optim.SGD(params_to_update, lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(resnet_ffe, train_loader, criterion, optimizer)
            validate_model(resnet_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(resnet_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 100x magnification

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_100x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    resnet_ft = create_model('ResNet', 2)
    resnet_ft = resnet_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(resnet_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(resnet_ft, train_loader, criterion, optimizer)
            validate_model(resnet_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(resnet_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    resnet_ffe = create_model('ResNet', 2, is_fixed_feature_extractor=True)
    resnet_ffe = resnet_ffe.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)
    params_to_update = []
    for name,param in resnet_ffe.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)

    optimizer = optim.SGD(params_to_update, lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(resnet_ffe, train_loader, criterion, optimizer)
            validate_model(resnet_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(resnet_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
E

### Working with images at 200x magnification

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_200x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    resnet_ft = create_model('ResNet', 2)
    resnet_ft = resnet_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(resnet_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(resnet_ft, train_loader, criterion, optimizer)
            validate_model(resnet_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(resnet_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    resnet_ffe = create_model('ResNet', 2, is_fixed_feature_extractor=True)
    resnet_ffe = resnet_ffe.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)
    params_to_update = []
    for name,param in resnet_ffe.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)

    optimizer = optim.SGD(params_to_update, lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(resnet_ffe, train_loader, criterion, optimizer)
            validate_model(resnet_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(resnet_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 400x magnification

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_400x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    resnet_ft = create_model('ResNet', 2)
    resnet_ft = resnet_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(resnet_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(resnet_ft, train_loader, criterion, optimizer)
            validate_model(resnet_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(resnet_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    resnet_ffe = create_model('ResNet', 2, is_fixed_feature_extractor=True)
    resnet_ffe = resnet_ffe.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)
    params_to_update = []
    for name,param in resnet_ffe.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)

    optimizer = optim.SGD(params_to_update, lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(resnet_ffe, train_loader, criterion, optimizer)
            validate_model(resnet_ffe, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(resnet_ffe, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

## Vision Transformer models

Data Augmentation specific to models

In [11]:
data_transforms = {
    'train_transform' : transforms.Compose(
        [
            transforms.Resize(224),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ]
    ),

    'test_transform' : transforms.Compose(
        [
            transforms.TenCrop(224),
            transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
        ]
    )
}

### Working with images at 40x magnification

In [12]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_40x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

fine tuning the model

In [13]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    transformer_ft = create_model('Transformer', 2)
    transformer_ft = transformer_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(transformer_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(transformer_ft, train_loader, criterion, optimizer)
            validate_model(transformer_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(transformer_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 100x magnification 

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_100x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    transformer_ft = create_model('Transformer', 2)
    transformer_ft = transformer_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(transformer_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(transformer_ft, train_loader, criterion, optimizer)
            validate_model(transformer_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(transformer_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

### Working with images at 200x magnification

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_200x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    transformer_ft = create_model('Transformer', 2)
    transformer_ft = transformer_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(transformer_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(transformer_ft, train_loader, criterion, optimizer)
            validate_model(transformer_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(transformer_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch

In [ ]:
data_dir = '/content/drive/MyDrive/UKZN/Comp700/Normalized_400x'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x)
                                          ) for x in ['testing', 'training']}

data = image_datasets['training']
data2 = image_datasets['testing']

In [ ]:
num_splits = 10
criterion = nn.CrossEntropyLoss()
splits = KFold(n_splits=num_splits, shuffle=True, random_state=42)
model_performance = {
    'accuracy': [],
    'sensitivity': [],
    'precision': [],
    'specificity': [],
    'F1': [],
}
test_loader = DataLoader(WrapperDataset(data2, transform=data_transforms['test_transform']), batch_size=16)

for fold, (train_index, valid_index) in enumerate(splits.split(np.arange(len(data)))):
    print('Fold {}'.format(fold + 1))

    transformer_ft = create_model('Transformer', 2)
    transformer_ft = transformer_ft.to(device)
    train_sampler = SubsetRandomSampler(train_index)
    val_sampler = SubsetRandomSampler(valid_index)
    train_loader = DataLoader(WrapperDataset(data, transform=data_transforms['train_transform']), batch_size=16, sampler=train_sampler)
    val_loader = DataLoader(WrapperDataset(data, transform=data_transforms['test_transform']), batch_size=16, sampler=val_sampler)

    optimizer = optim.SGD(transformer_ft.parameters(), lr=0.0001, momentum=0.9) # as specified in literature
    for epoch in range(10):
            train_model(transformer_ft, train_loader, criterion, optimizer)
            validate_model(transformer_ft, val_loader, criterion)
            print("Epoch:{}/{}".format(epoch+1, 10))
    
    accuracy, sensitivity, precision, specificity, F1 = model_eval(transformer_ft, test_loader)
    model_performance['accuracy'].append(accuracy)
    model_performance['sensitivity'].append(sensitivity)
    model_performance['precision'].append(precision)
    model_performance['specificity'].append(specificity)
    model_performance['F1'].append(F1)

print('MODEL MEAN PERFORMANCE: ')
print(f"Mean Accuracy: {np.average(model_performance['accuracy'])}")
print(f"Mean Sensitivity: {np.average(model_performance['sensitivity'])}")
print(f"Mean Precision: {np.average(model_performance['precision'])}")
print(f"Mean Specificity: {np.average(model_performance['specificity'])}")
print(f"Mean F1-Score: {np.average(model_performance['F1'])}")

Fold 1
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 2
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 3
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 4
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 5
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 6
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 7
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 8
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch:5/10
Epoch:6/10
Epoch:7/10
Epoch:8/10
Epoch:9/10
Epoch:10/10
Fold 9
Epoch:1/10
Epoch:2/10
Epoch:3/10
Epoch:4/10
Epoch